<a href="https://colab.research.google.com/github/GenoKiller777/ML-Notebooks/blob/main/MLFlow_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalacion de MLFlow
!pip install mlflow

In [ ]:
!pip install pyngrok

In [2]:
#Checkear si esta instalado
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [3]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment('AprendiendoMLFlow-Demo')

2023/10/25 01:09:01 INFO mlflow.tracking.fluent: Experiment with name 'AprendiendoMLFlow-Demo' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/858518853539034825', creation_time=1698196141699, experiment_id='858518853539034825', last_update_time=1698196141699, lifecycle_stage='active', name='AprendiendoMLFlow-Demo', tags={}>

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import sys
import os

In [5]:
def eval_metrics(actual,pred):
  rmse = np.sqrt(mean_squared_error(actual,pred))
  mae = mean_absolute_error(actual,pred)
  r2 = r2_score(actual,pred)
  return rmse, mae, r2

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv', delimiter = ";")

In [8]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
def  train_model(alpha,l1_ratio):
  #Divida los datos en conjuntos de entrenamiento y prueba. (0,75, 0,25)
    train, test = train_test_split(data)

  # La columna prevista es "calidad", que es un escalar de [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    with mlflow.start_run():
      lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
      lr.fit(train_x, train_y)

      predicted_qualities = lr.predict(test_x)

      (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

      print(f"Elasticnet model (alpha={alpha:f}, l1_ratio={l1_ratio:f}):")
      print(f"  RMSE: {rmse}")
      print(f"  MAE: {mae}")
      print(f"  R2: {r2}")

      mlflow.log_param("alpha", alpha)
      mlflow.log_param("l1_ratio", l1_ratio)
      mlflow.log_metric("rmse", rmse)
      mlflow.log_metric("r2", r2)
      mlflow.log_metric("mae", mae)

      mlflow.sklearn.log_model(lr,"model")

In [10]:
train_model(0.0,0.0)

<ipython-input-9-0f95dd3655a2>:13: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lr.fit(train_x, train_y)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.485e+02, tolerance: 7.698e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Elasticnet model (alpha=0.000000, l1_ratio=0.000000):
  RMSE: 0.6570387140569496
  MAE: 0.503659708955498
  R2: 0.3646742567247895


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [11]:
train_model(0.5,0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.719738192880925
  MAE: 0.587624284578391
  R2: 0.15480002236661827


In [12]:
train_model(0.75,0.75)

Elasticnet model (alpha=0.750000, l1_ratio=0.750000):
  RMSE: 0.806272018389023
  MAE: 0.656285759442514
  R2: 0.02522931828293762


In [13]:
train_model(1,1)

Elasticnet model (alpha=1.000000, l1_ratio=1.000000):
  RMSE: 0.7980834278335203
  MAE: 0.6571988057748842
  R2: 0.030454041487567807


In [14]:
train_model(0.75,1)

Elasticnet model (alpha=0.750000, l1_ratio=1.000000):
  RMSE: 0.7481893223563713
  MAE: 0.6323358657164262
  R2: 0.055121668364151


In [15]:
train_model(1,0.75)

Elasticnet model (alpha=1.000000, l1_ratio=0.750000):
  RMSE: 0.7938979143575715
  MAE: 0.6641271453002272
  R2: 0.04180928368925263


In [16]:
train_model(1.0,0.5)

Elasticnet model (alpha=1.000000, l1_ratio=0.500000):
  RMSE: 0.7677877391008325
  MAE: 0.638092010155586
  R2: 0.025371408213165414


In [22]:
from pyngrok import ngrok
#Terminar túneles abiertos si existen
ngrok.kill()

#Configuración del token de autenticación (opcional)
#Obtén tu token de autenticación https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2XEec4hrhYjLwSGCx8oeWfA5vKl_2Tuwtzko2XNnUL6op3SS8"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

#Abrir un túnel HTTPs en el puerto 5000 para http://localhost:5000
ngrok_tunnel = ngrok.connect(addr ="5000", proto = "http", bind_tls = True)
print("Seguimiento UI MLFlow:", ngrok_tunnel.public_url)

Seguimiento UI MLFlow: https://501b-34-171-255-117.ngrok-free.app


In [ ]:
!mlflow ui